In [105]:
import pyspark 
import argparse 
import json 

# 4.1.1 B: The number of distinct users who have written the reviews

sc_conf = pyspark.SparkConf()
sc_conf.setAppName('task1')  
sc_conf.setMaster('local[*]') 
sc_conf.set('spark.driver.memory', '8g')
sc_conf.set('spark.executor.memory', '4g')
# sc = pyspark.SparkContext(conf=sc_conf)
sc = pyspark.SparkContext.getOrCreate()
sc.setLogLevel("OFF")

# read in the lines and create the rdd 
lines = sc.textFile("./data/review.json")

# loading the json into a python object 
rdd = lines.map(lambda line: json.loads(line))

Get all unique users 

create tuple (user_id, 1) for each unique review 
Create a count from the tuples     
then sort by greatest count 

then do a select on the top m users 

In [123]:
unique_users = rdd.map(lambda x: (x['user_id'], x['review_id'])) 

In [124]:
unique_users = unique_users.distinct().collect() # gives us a list of unique users 


In [125]:
unique_users[0] # user_id, review_id 

('86J5DwcFk4f4In1Vxe2TvA', '1wVA2-vQIuW_ClmXkDxqMQ')

In [109]:
unique_reviews = rdd.map(lambda x: [x['review_id'], x['user_id']]) # gives us [[review_id, user_id]]

In [122]:
unique_reviews = unique_reviews.distinct()
print(type(unique_reviews))


<class 'pyspark.rdd.PipelinedRDD'>


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 13 in stage 129.0 failed 1 times, most recent failure: Lost task 13.0 in stage 129.0 (TID 1227) (172.27.118.185 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/spark/spark-3.3.1-bin-hadoop3/python/lib/pyspark.zip/pyspark/worker.py", line 686, in main
    process()
  File "/opt/spark/spark-3.3.1-bin-hadoop3/python/lib/pyspark.zip/pyspark/worker.py", line 676, in process
    out_iter = func(split_index, iterator)
  File "/home/lucky/miniconda3/lib/python3.10/site-packages/pyspark/rdd.py", line 3472, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/home/lucky/miniconda3/lib/python3.10/site-packages/pyspark/rdd.py", line 3472, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/home/lucky/miniconda3/lib/python

In [111]:
unique_users = sc.parallelize(unique_users)
unique_reviews = sc.parallelize(unique_reviews)


In [112]:
print(unique_users.take(1))
print("review_id, user_id")
print(unique_reviews.take(1))

[('-mA3-1mN4JIEkqOtdbNXCQ', 1)]
review_id, user_id
[['-I5umRTkhw15RqpKMl_o1Q', '-mA3-1mN4JIEkqOtdbNXCQ']]


In [113]:
print(unique_users.take(1)[0][0])
print(unique_reviews.take(1)[0][1])

-mA3-1mN4JIEkqOtdbNXCQ
-mA3-1mN4JIEkqOtdbNXCQ


In [114]:
review_id_and_user_id = unique_users.join(unique_reviews).distinct()

In [115]:
review_id_and_user_id.take(4) # review_id, ()

[('wLofLemhls9oQqFpc4qEQQ', (1, 'gmfckUvQZQmcPm8RGRiwOQ')),
 ('Ml5Q1rLjyUmS-Q2dQ2C4nw', (1, '-i7iRw_DI_T0wTSXsfiY8Q')),
 ('dEk9YPjigYm8_7f9H8t_EQ', (1, 'byNMn9Hf9cv_Amj2_5ydow')),
 ('b9-9XmpCefla_r-6-Dm0vg', (1, 'YvqDqcjAvCvs-H1IU7viLg'))]

In [116]:
review_id_and_user_id = review_id_and_user_id.map(lambda x, y: (x, 1))
print(type(review_id_and_user_id))

<class 'pyspark.rdd.PipelinedRDD'>


In [117]:
review_id_and_user_id.collect()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 7 in stage 124.0 failed 1 times, most recent failure: Lost task 7.0 in stage 124.0 (TID 1155) (172.27.118.185 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/spark/spark-3.3.1-bin-hadoop3/python/lib/pyspark.zip/pyspark/worker.py", line 686, in main
    process()
  File "/opt/spark/spark-3.3.1-bin-hadoop3/python/lib/pyspark.zip/pyspark/worker.py", line 678, in process
    serializer.dump_stream(out_iter, outfile)
  File "/opt/spark/spark-3.3.1-bin-hadoop3/python/lib/pyspark.zip/pyspark/serializers.py", line 273, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/opt/spark/spark-3.3.1-bin-hadoop3/python/lib/pyspark.zip/pyspark/util.py", line 81, in wrapper
    return f(*args, **kwargs)
TypeError: <lambda>() missing 1 required pos

In [ ]:
temp = review_id_and_user_id.collect()

In [ ]:
print(temp[0])

('wLofLemhls9oQqFpc4qEQQ', (1, 'gmfckUvQZQmcPm8RGRiwOQ'))


In [ ]:
rdd = sc.parallelize([1, 1, 2, 3, 5, 8])

In [ ]:
result = rdd.groupBy(lambda x: x % 2).collect()
print(type(result))

<class 'list'>
